## Check the setup and connect to the database

In [ ]:
%run "../01-check_setup.ipynb"

In [ ]:
myconn.get_tables(schema='NHTSA')

# Car complaints data

The __National Highway Traffic Safety Administration (NHTSA)__, is part of the U.S. Department of Transportation.
Complaint information entered into NHTSA’s Office of Defects Investigation __vehicle owner's complaint database__ is used with other data sources to identify __safety issues__ that warrant investigation and to determine if a safety-related defect trend exists. Complaint information is also analyzed to monitor existing recalls for proper scope and adequacy. The NHTSA provides a large dataset of complaints related to cars in the US:[https://www.nhtsa.gov/nhtsa-datasets-and-apis#complaints].
For this demo scenario, we've loaded the 2024 complaints data, for detail instructions see appendix.


In [ ]:
# Create a HANA dataframe for the complaints data, pre-loaded into SAP HANA Cloud
hdf_complaints=myconn.table('COMPLAINTS', schema='NHTSA')
# hdf_complaints.get_table_structure()

In [ ]:
# Overview the complaints data
import pandas as pd
pd.set_option('max_colwidth', None) 
display(
    hdf_complaints.filter("""PROD_TYPE='V'""") # Vehicle-related complaint
    .select('CMPLID', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'COMPDESC', 'CDESCR')
    .head(1).collect().T
)

In [ ]:
# Let's filter on specific component-groups, for detailed classification analysis
hdf_carcomplaints=(hdf_complaints
    .select('CMPLID', 'MFR_NAME', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'FUEL_TYPE', 'CRASH', 'FIRE', 'STATE', 'CMPL_TYPE',
            'ANTI_BRAKES_YN', 'CRUISE_CONT_YN', 'DRIVE_TRAIN', 'VEHICLES_TOWED_YN', 'CDESCR', 'COMPDESC')
    .filter('''COMPDESC IN ('AIR BAGS','ELECTRICAL SYSTEM', 'SERVICE BRAKES','STEERING')'''))

hdf_carcomplaints.count()

In [ ]:
pd.set_option('max_colwidth', None) 
display(hdf_carcomplaints.head(1).collect().T)

## Text splitting, preparing complaints description text for vectorization

Text embedding models and Large Language Models (LLMs) often have token length limits, hence managing the text length before running it through such models is a frequent preprocessing task. For that purpose, a new text splitting function __hana_ml.text.text_splitter__ is being introduced and explained in more detail in the following blog post: [Text chunking - an exciting new NLP function in SAP HANA Cloud](https://community.sap.com/t5/technology-blogs-by-sap/text-chunking-an-exciting-new-nlp-function-in-sap-hana-cloud/ba-p/13958766).

In [ ]:
# Determining the character length (using SQL length-fct) of a given text. For western languages, character-length / 3 or 4 is giving an approximate token length
# Note, the text analysis function applied above, also determines the token length specifically

hdf_carcomplaints.select('CMPLID', ('LENGTH("CDESCR")', 'LEN_CDESCR')).sort('LEN_CDESCR', desc=True).head(100).collect()

In [ ]:
hdf_carcomplaints.filter("""CMPLID=1918524""").select("CDESCR").collect().T

In [ ]:
# Applying the Text Splitter with recursive-splitting, available with hana-ml 2.23
from hana_ml.text.text_splitter import TextSplitter

splitter = TextSplitter(split_type='recursive', chunk_size=512, overlap=64)
splitter._extend_pal_parameter({'GLOBAL_SEPARATOR':'[.]', 'KEEP_SEPARATOR':1})
splitted_text = splitter.split_text(
    hdf_carcomplaints.filter("LENGTH(CDESCR)>=512").select('CMPLID', 'CDESCR').head(10), 
    order_status=True
    )
#print(splitted_text.shape)
display(splitter.statistics_.collect())

display(splitted_text.select("*", ('LENGTH("CONTENT")', "CHUNK_SIZE")).head(15).collect())

## Generating Text Embeddings in SAP HANA Cloud

SAP HANA Cloud introduces availability of a text embedding model, targeted for vectorization of text data already stored within the database. It is specifically useful for vector engine similarity search scenarios or machine learning tasks, implicitly making use text embedding vectors unlocking the semantic understanding of text data for the analysis. A detailed capability introduction can be found in the following blog post [Text Embedding Service in SAP HANA Cloud Predictive Analysis Library (PAL)](https://community.sap.com/t5/enterprise-resource-planning-blogs-by-sap/text-embedding-service-in-sap-hana-cloud-predictive-analysis-library-pal/ba-p/13958864).

Running vectorizations for 122447 recs took 37 minutes (2244 secs) on 4 vCPUs.

For the sake of the exercise let's reduce the number of records by limiting only to cars with hybrid engine.

In [ ]:
print(f"""Number of records selected for further processing: {(hdf_carcomplaints_he := hdf_carcomplaints.filter(''' "FUEL_TYPE"='HE' ''')).count()}""")

In [ ]:
### Generating Text Embeddings in SAP HANA Cloud with the new PAL function, function available with hana-ml 2.23.
from hana_ml.text.pal_embeddings import PALEmbeddings
pe = PALEmbeddings(model_version='SAP_GXY.20250407')
textembeddings = pe.fit_transform(hdf_carcomplaints_he, key="CMPLID", target=["CDESCR"], thread_number=10, batch_size=10) #, max_token_num=512
print(f"{textembeddings.count()} records processed in {round(pe.runtime, 3)} sec")

In [ ]:
textembeddings.get_table_structure()

In [ ]:
# Review the generated Text Embeddings, note they are of vector dimensionality 768 for the current HANA embedding model
cmpl_textembeddings=textembeddings.rename_columns({'VECTOR_COL_CDESCR': 'HANACLOUD_TEXT_EMBEDDING'}).to_tail('COMPDESC')
display(cmpl_textembeddings.head(1).collect().T)

## Text Embeddings for Advanced Text Mining using ANNS (Approximate Nearest Neighbor Search)

New text mining techniques can now be applied leveraging existing text embedding vectors (or create them implicitly from the original text) unlocking the semantic understanding of text data for text mining tasks like text document classification. Thus elevating a classic task from linguistic text understanding to understanding the real natural language context of documents for greatly improved results. This new technique in SAP HANA Cloud Predictive Analysis Library for advanced text mining builds on the concept of approximate neares neighbor search, thus also adressing larger scale scenarios. The advanced text mining function is explained with more detail in the following blog post [New information retrieval techniques in SAP HANA Cloud using BM25 and ANNS for Advanced Text Mining](https://community.sap.com/t5/technology-blogs-by-sap/new-information-retrieval-techniques-in-sap-hana-cloud-using-bm25-and-anns/ba-p/13958729).

In [ ]:
hdf_carcomplaints.filter('''MFR_NAME LIKE ('Tesla%')''').agg([('count', 'YEARTXT', 'COUNT')], group_by='YEARTXT').sort(cols='YEARTXT', desc=False).collect()

In [ ]:
# Preparing the input dataframe for ANNS-advanced text mining
# Required input data schema: document-id, text
carcomplaints_anns_hdf=hdf_carcomplaints.filter('''MFR_NAME LIKE ('Tesla%') AND "YEARTXT"='2022' ''').select('CMPLID', 'CDESCR')
display(carcomplaints_anns_hdf.head(1).collect())
display(carcomplaints_anns_hdf.count())

In [ ]:
# Building the ANNS-advanced text mining model
# Note, the complaints shall be clustered into 70 groups for the approximate similarity search
from hana_ml.text.anns_model import ANNSModel

anns = ANNSModel(by_doc=True)
anns.fit(data=carcomplaints_anns_hdf, key='CMPLID', target='CDESCR', group_number=70, max_iteration=100, 
         comment='TESLA complaints search model for 2022')

print(f"Runtime building the ANN-text mining search model：{anns.runtime} s")

In [ ]:
# Reviewing the implicit embedding generation results from the ANNS-advanced text mining model building
print(anns.embedding_result_.shape)
display(anns.embedding_result_.head(1).collect().T)
print(f"""Vector cardinality: {anns.embedding_result_.head(1).select(("CARDINALITY(VECTOR_COL)","A")).collect()["A"][0]}""")

In [ ]:
# The ANNS-advanced text mining model is kept as a stateful in-memory model for fastest search predictions
display(anns.state_.collect().T)

In [ ]:
# Now, search for CDESCR complaints with the search text: '12v battery is empty'
# Prepare the search query dataframe as: Searchquery-Id, text
hdf_anns_query=myconn.sql(
    '''Select '999' as "QID", 'battery exploded' as "SEARCHTEXT" from "DUMMY"
    UNION ALL
    Select '998' as "QID", '12v battery empty' as "SEARCHTEXT" from "DUMMY"'''
    )
hdf_anns_query.collect()

In [ ]:
# Execute the adv. text mining / ANN search 
res = anns.predict(data=hdf_anns_query, key='QID', target='SEARCHTEXT', is_query=True, k_nearest_neighbours=3, k_cluster=70) 
print(f"Runtime ：{anns.runtime} s") 
display(res.collect()) 

In [ ]:
# Joining the adv.text mining/ann search results with the original CDESCR-text
anns_top_searchresults=(
    res.rename_columns({'TEST_QID': 'QID'})
    .set_index("TRAIN_ID")
    .join(carcomplaints_anns_hdf.set_index("CMPLID"))
    )
display(anns_top_searchresults.rename_columns({'TRAIN_ID': 'CMPLID'}).sort('DISTANCE').collect())

In [ ]:
# List the stateful Adv.text mining/ANNS-models on the system
from hana_ml.text.anns_model import list_models
list_models(myconn).collect()

In [ ]:
# Delete last <anns-object>-model instance
anns.delete_model().collect().T

In [ ]:
# List the stateful Adv.text mining/ANNS-models on the system, again
annsmodelstodelete=list_models(myconn)
annsmodelstodelete.collect()